# AI Platform - Pipeline for Training and Serving

This notebook creates a python package for a TensorFlow training project that uses the `<PROJECT_ID>.digits.digits_prepped` BigQuery table. AI Platform clients are used to setup an AI Platform custom training pipeline that runs the training job and uploads the resulting model.  Then AI Platform clients are used to deploy the model to an endpoint for online predictions.

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`
- `05 - AI Platform - Training Job and Serving` 
    - this notebook uses the python package created in 05
    
**Resources**
- Based on:
    - https://cloud.google.com/ai-platform-unified/docs/training/create-training-pipeline#custom-job-model-upload
- Using PipelineService:
    - https://googleapis.dev/python/aiplatform/latest/aiplatform_v1beta1/pipeline_service.html

**Overview**

<img src="architectures/statmike-mlops-06.png">

---
## Setup

Setup the environment:

In [1]:
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from datetime import datetime

Define Parameters:

In [2]:
# Locations
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
BUCKET_NAME='gs://{}/digits/aip_train_job'.format(PROJECT_ID)
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
JOB_NAME='AIP_DIGITS_'+TIMESTAMP
MODEL_DIR = '{}/{}'.format(BUCKET_NAME, JOB_NAME)
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

# Resources
TRAIN_IMAGE='us-docker.pkg.dev/cloud-aiplatform/training/tf-cpu.2-4:latest'
DEPLOY_IMAGE ='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-3:latest'
TRAIN_COMPUTE='n1-standard-4'
DEPLOY_COMPUTE='n1-standard-4'

# TF Parameters to pass
EPOCHS = 25
BATCH_SIZE = 30

Setup AI Platform Python Clients
- https://googleapis.dev/python/aiplatform/latest/index.html

In [3]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

---
## Custom Pipeline Job

Create a client for the AIP Pipeline Service:

In [4]:
clients['pipeline'] = aiplatform.gapic.PipelineServiceClient(client_options=client_options)

Define Training Job Parameters - to match the yaml spec in
- `gs://google-cloud-aiplatform/schema/trainingjob/definition/custom_task_1.0.0.yaml`

In [5]:
MACHINE_SPEC = {
    "machineType": TRAIN_COMPUTE,
    "acceleratorCount": 0
}


CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE)
]

WORKER_POOL_SPEC = [
    {
        "replicaCount": 1,
        "machineSpec": MACHINE_SPEC,
        "pythonPackageSpec": {
            "executorImageUri": TRAIN_IMAGE,
            "packageUris": [BUCKET_NAME + "/trainer_cifar.tar.gz"],
            "pythonModule": "trainer.task",
            "args": CMDARGS
        }
    }
]

JOB_SPEC = {
    "workerPoolSpecs": WORKER_POOL_SPEC,
    "baseOutputDirectory": {"outputUriPrefix": MODEL_DIR}
    
}

CUSTOM_JOB = {
    "display_name": JOB_NAME,
    "job_spec": JOB_SPEC
}

Define Training Pipeline Parameters:

In [6]:
training_task_definition = "gs://google-cloud-aiplatform/schema/trainingjob/definition/custom_task_1.0.0.yaml"
training_task_inputs = json_format.ParseDict(JOB_SPEC, Value())

training_pipeline = {
    "display_name": JOB_NAME,
    "training_task_definition": training_task_definition,
    "training_task_inputs": training_task_inputs,
    "model_to_upload": {
        "display_name": JOB_NAME,
        "container_spec": {"image_uri": DEPLOY_IMAGE},
    },
}

Submit pipeline job:
- this create a pipeline job
    - which create a custom training job
    - and when complete, uploads a model

In [7]:
pipeline = clients['pipeline'].create_training_pipeline(parent=PARENT, training_pipeline=training_pipeline)

In [9]:
clients['pipeline'].get_training_pipeline(name=pipeline.name).state

<PipelineState.PIPELINE_STATE_SUCCEEDED: 4>

In [10]:
clients['pipeline'].get_training_pipeline(name=pipeline.name).model_to_upload.name

'projects/691911073727/locations/us-central1/models/2088491550634934272'

---
## Endpoint Creation

Create a client to the endpoint service:

In [11]:
clients['endpoint'] = aiplatform.gapic.EndpointServiceClient(client_options=client_options)

Create the endpoint:

In [12]:
ENDPOINT_NAME = 'ENDPOINT_'+JOB_NAME
endpoint = clients['endpoint'].create_endpoint(parent=PARENT, endpoint={"display_name": ENDPOINT_NAME})

In [13]:
endpoint_info = clients['endpoint'].get_endpoint(name=endpoint.result(timeout=180).name)
endpoint_info.name

'projects/691911073727/locations/us-central1/endpoints/4761589035711856640'

---
## Deploy Model to Endpoint

Setup Deployment Parameters:

In [14]:
MACHINE_SPEC = {
    "machine_type": DEPLOY_COMPUTE,
    "accelerator_count": 0,
}
DMODEL = {
        "model": clients['pipeline'].get_training_pipeline(name=pipeline.name).model_to_upload.name,
        "display_name": 'DEPLOYED_'+JOB_NAME,
        "dedicated_resources": {
            "min_replica_count": 1,
            "max_replica_count": 2,
            "machine_spec": MACHINE_SPEC
        }   
}
TRAFFIC = {
    '0' : 100
}

Deploy the Model to the Endpoint:

In [15]:
dmodel = clients['endpoint'].deploy_model(endpoint=endpoint_info.name, deployed_model=DMODEL, traffic_split=TRAFFIC)

In [16]:
dmodel_info = dmodel.result().deployed_model
dmodel_info.id

'3490940622095777792'

In [17]:
clients['endpoint'].get_endpoint(name=endpoint_info.name)

name: "projects/691911073727/locations/us-central1/endpoints/4761589035711856640"
display_name: "ENDPOINT_AIP_DIGITS_20210413150956"
deployed_models {
  id: "3490940622095777792"
  model: "projects/691911073727/locations/us-central1/models/2088491550634934272"
  display_name: "DEPLOYED_AIP_DIGITS_20210413150956"
  create_time {
    seconds: 1618327452
    nanos: 50865000
  }
  dedicated_resources {
    machine_spec {
      machine_type: "n1-standard-4"
    }
    min_replica_count: 1
    max_replica_count: 2
  }
}
traffic_split {
  key: "3490940622095777792"
  value: 100
}
etag: "AMEw9yOXC8JF4Y2F3TKP-K3jNVh60YcvSvLIC0QclbNlwm_BV2D7PBlVWV6wDi73C8ur"
create_time {
  seconds: 1618327351
  nanos: 490411000
}
update_time {
  seconds: 1618327922
  nanos: 748729000
}

---
## Prediction

Create a client to the prediction service:

In [18]:
clients['prediction'] = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

Setup an observation for prediction:

In [19]:
%%bigquery pred
SELECT *
FROM `digits.digits_prepped`
WHERE splits='TEST'

In [20]:
pred.head(1)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p57,p58,p59,p60,p61,p62,p63,target,target_OE,SPLITS
0,0.0,0.0,0.0,10.0,11.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,15.0,14.0,5.0,0.0,0,Even,TEST


In [21]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')[0]
#newob

In [22]:
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

response = clients['prediction'].predict(endpoint=endpoint_info.name, instances=[json_format.ParseDict(newob, Value())], parameters=json_format.ParseDict({}, Value()))

In [23]:
response.predictions

[[0.415978789, 0.000595294638, 1.73271346e-05, 6.54000189e-07, 1.201748e-05, 8.78807214e-07, 0.583141863, 1.45979229e-09, 3.11378426e-05, 0.000222129267]]

In [24]:
import numpy as np
np.argmax(response.predictions[0])

6

---

## Remove Resources
- undeploy model
- delete endpoint
- remove model
- delete training > custom job
- delete training > training pipeline
- delete python training application
    - these could be used by later notebooks in this project

Get Pipeline Info:

In [34]:
pipe_info = clients['pipeline'].get_training_pipeline(name=pipeline.name)

Undeploy Model:

In [26]:
dmodel = clients['endpoint'].get_endpoint(name=endpoint_info.name).deployed_models[0].id
clients['endpoint'].undeploy_model(endpoint=endpoint_info.name, deployed_model_id=dmodel)

Delete Endpoint:

In [27]:
clients['endpoint'].delete_endpoint(name=endpoint_info.name)

Remove Model:

In [36]:
clients['model'] = aiplatform.gapic.ModelServiceClient(client_options=client_options)
clients['model'].delete_model(name=pipe_info.model_to_upload.name)

Delete Training > Custom Job:

In [46]:
clients['job'] = aiplatform.gapic.JobServiceClient(client_options=client_options)
clients['job'].delete_custom_job(name=pipe_info.training_task_metadata._pb['backingCustomJob'].string_value)

Delete Training > Training Pipeline:

In [49]:
clients['pipeline'].delete_training_pipeline(name=pipeline.name)

Delete Model Files:
- This includes the training package and the saved model
- These could be useful for other notebooks

In [50]:
from google.cloud import storage
gcs = storage.Client()

path = gcs.bucket(PROJECT_ID)
blobs = path.list_blobs(prefix='digits/aip_train_job')
for blob in blobs:
    print(blob)
    blob.delete()

<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413150956/, 1618327171620676>
<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413150956/model/, 1618327171918285>
<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413150956/model/assets/, 1618327175253828>
<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413150956/model/saved_model.pb, 1618327175719390>
<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413150956/model/variables/, 1618327172067587>
<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413150956/model/variables/variables.data-00000-of-00001, 1618327174237522>
<Blob: statmike-mlops, digits/aip_train_job/AIP_DIGITS_20210413150956/model/variables/variables.index, 1618327174494912>
<Blob: statmike-mlops, digits/aip_train_job/trainer_cifar.tar.gz, 1618322304382464>
